In [ ]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [ ]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

やりたいこと<br>
・Samedeviceマッチングによって、smn_uidが不明なadvertising_idにsmn_uidを紐づけし、アプリ配信量を増やす<br>

2019年6月5にもらったDrawbridgeリストでidfaを格納したリスト

In [ ]:
db_idfa_table = "segment.drawbridge_idfas" 
print(db_idfa_table)

2019年6月5日にもらったDrawbridgeリストでcookie_idを格納したリスト

In [ ]:
db_cookie_table = "segment.drawbridge_smn_uids" 
print(db_cookie_table)

In [ ]:
query = ( 
    f"""
    select
    count(smn_uid) as uu_smn_uid
    from 
    {db_cookie_table}
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_uid = ir.sql_to_pandas(query)

In [ ]:
df_uid.head()/1000000 #1.7億レコード

In [ ]:
query = ( 
    f"""
    select
    count(advertising_id) as uu_ad_id
    from 
    {db_idfa_table}
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_idfa = ir.sql_to_pandas(query)

In [ ]:
df_idfa.head()/1000000 #4500万レコード

In [ ]:
query = ( 
    f"""
    select
    count(distinct t1.drawbridge_consumer_id) as cnt_db_id
    from 
    {db_idfa_table} t1
    inner join 
    {db_cookie_table} t2
    on t1.drawbridge_consumer_id = t2.drawbridge_consumer_id
    where 
    smn_uid > 0 
    and advertising_id is not null
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_tmp = ir.sql_to_pandas(query)

In [ ]:
df_tmp.head()/1000000

↑smn_uidとadvertising_id両方を持っているユーザーは約1300万ユーザー

In [ ]:
#両IDを持っている（セイムデバイスの可能性のある）Drawbridge_idリストを生成
db_list = f"dm_tmp.samedevice_userlist_190723"
print(db_list)

In [ ]:
query = ( 
    f"""
    select
    distinct t1.drawbridge_consumer_id
    from 
    {db_idfa_table} t1
    inner join 
    {db_cookie_table} t2
    on t1.drawbridge_consumer_id = t2.drawbridge_consumer_id
    where 
    smn_uid > 0 
    and advertising_id is not null
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_db = ir.sql_to_pandas(query)
        ir.upload_dataframe(db_list, df_db, ['string'])

紐づけが可能そうなadvertising_idリストを作成する

In [ ]:
adid_list = f"segment.ya_samedevice_adid_userlist_190723"
print(adid_list)

ファイルが重くなるので、分割して実施して作成

In [ ]:
%%slack_notify @yusuke_akada
temp_table = "dm_tmp.ya_temp_samedevice_190723"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #まず空tableを作る
    query = (
        f"""
        create table {adid_list}
            (
            drawbridge_consumer_id string,
            advertising_id string,
            type string
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query)    

In [ ]:
# テキストファイル単体ごとに読込、DataFrame化
for i in range(0,200):
    # 親テーブルにInsert
    query = (
        f"""
        insert overwrite table {adid_list}
        (
        drawbridge_consumer_id,
        advertising_id,
        type
        )

        partition
        (
        block = {i}
        )
        
        select
        drawbridge_consumer_id,
        advertising_id,
        type
        from 
        segment.drawbridge_idfas
        where
        block = {i} 
        and drawbridge_consumer_id in (select drawbridge_consumer_id from {db_list})
                
        """
        )
    ir.execute_query_no_ret(query) 

In [ ]:
query = ( 
    f"""
    select
    count(advertising_id) as ad_id
    from 
    {adid_list}
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_adid = ir.sql_to_pandas(query)

In [ ]:
df_adid/1000000

In [ ]:
#↑セイムデバイスの可能性のあるadvertising_id = 36852179

紐づけが可能そうなsmn_uidリストを作成する

In [ ]:
uid_list = f"segment.ya_samedevice_smn_uid_userlist_190723"
print(uid_list)

ファイルが重くなるので、分割して実施する

In [ ]:
%%slack_notify @yusuke_akada
temp_table = "dm_tmp.ya_temp_samedevice_190723"    # 分割処理用のtemporary table 実行後dropされる


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #まず空tableを作る
    query = (
        f"""
        create table {uid_list}
            (
            drawbridge_consumer_id string,
            smn_uid bigint,
            type string
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query)    

In [ ]:
%%slack_notify @yusuke_akada
# テキストファイル単体ごとに読込、DataFrame化
for i in range(0,200):
    # 親テーブルにInsert
    query = (
        f"""
        insert overwrite table {uid_list}
        (
        drawbridge_consumer_id,
        smn_uid,
        type
        )

        partition
        (
        block = {i}
        )
        
        select
        drawbridge_consumer_id,
        smn_uid,
        type
        from 
        segment.drawbridge_smn_uids
        where
        block = {i} 
        and drawbridge_consumer_id in (select drawbridge_consumer_id from {db_list})
                
        """
        )
    ir.execute_query_no_ret(query) 

In [ ]:
query = ( 
    f"""
    select
    count(smn_uid) as uid
    from 
    {uid_list}
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_uid = ir.sql_to_pandas(query)

In [ ]:
df_uid/1000000

In [ ]:
#↑セイムデバイスの可能性のあるsmn_uid = 117692826

In [ ]:
#とある一日分リクエストを参照
#smn_uidが不明なadvertising_idからのリクエスト数

In [ ]:
query = ( 
    f"""
    select 
    count(distinct advertising_id) as uu
    from dsplog.bid_response
    where 
    concat_ws('-',year,month,day) = '2019-07-22'
    and advertising_id is not null
    and smn_uid is null
    and user_agent_os_family in('Android','iOS')
    and is_app = True
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_adid = ir.sql_to_pandas(query)

In [ ]:
df_adid.head()/1000000

In [ ]:
#smn_uidが不明なadvertising_idからのリクエスト約2200万

In [ ]:
#Drawbridgeのリストでどれだけ判明できる余地があるか？

In [ ]:
query = ( 
    f"""
    select 
    count(distinct advertising_id) as uu
    from dsplog.bid_response
    where 
    concat_ws('-',year,month,day) = '2019-07-22'
    and advertising_id is not null
    and smn_uid is null
    and user_agent_os_family in('Android','iOS')
    and is_app = True
    and advertising_id in (select advertising_id from segment.ya_samedevice_adid_userlist_190723)
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_adid2 = ir.sql_to_pandas(query)

In [ ]:
df_adid2.head()/1000000

In [ ]:
#Drawbridgeのデータを使うことで判明できる候補は約570万ID →　25％

In [ ]:
df_adid2.head()/df_adid.head()

In [ ]:
#2.上記1のユーザーをsmn_uidベースとanidベースでbid_responseを参照する。

idfa_idベースで

In [ ]:
uid_table = "segment.ya_db_req_idfa_190723"   # idfaリストの最終的な対応table
temp_table = "dm_tmp.ya_temp_db_sample"    # 分割処理用のtemporary table 実行後drop

In [ ]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            advertising_id string
            ,smn_uid bigint
            ,drawbridge_consumer_id string
            ,user_agent_os_family string
            ,user_agent_device_family string
            ,user_agent_browser_family string
            ,user_agent_os_major string
            ,user_agent_os_minor string
            ,block int
            )
            
            partitioned by 
            (
            yymmdd string
            ,sample_size int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

In [ ]:
# 親テーブルにInsert
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    for i in range(0,200):
        for d in range(1,2):
            query = (
                f"""
                insert overwrite table {uid_table}
                (
                advertising_id,
                smn_uid,
                drawbridge_consumer_id,
                user_agent_os_family,
                user_agent_device_family,
                user_agent_browser_family,
                user_agent_os_major,
                user_agent_os_minor,
                block
                )
                partition
                (
                yymmdd = to_date(days_add(now(),-{d}))
                ,sample_size = {i}
                )
                select
                distinct t1.advertising_id
                ,t1.smn_uid
                ,drawbridge_consumer_id
                ,user_agent_os_family
                ,user_agent_device_family
                ,user_agent_browser_family
                ,user_agent_os_major
                ,user_agent_os_minor
                ,block
                from
                dsplog.bid_response t1
                inner join 
                segment.ya_samedevice_adid_userlist_190723 t2
                on t1.advertising_id = t2.advertising_id
                where 
                concat_ws('-',year,month,day) between to_date(days_add(now(),-{d})) and to_date(days_add(now(),-{d})) 
                and user_agent_os_family in('Android','iOS')
                and is_app = True
                and t1.advertising_id is not null
                and t1.smn_uid is null
                and block = {i}
                """
            )
            ir.execute_query_no_ret(query) 

In [ ]:
query = (
    f"""
    select
    count(distinct advertising_id) as adid_uu
    from 
    dm_tmp.ya_db_req_idfa
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_tmp_adid = ir.sql_to_pandas(query)

In [ ]:
df_tmp_adid.head()

cookie_idベースで

In [ ]:
uid_table = "segment.ya_db_req_smnuid_190723"   # idfaリストの最終的な対応table
temp_table = "dm_tmp.ya_temp_db_sample"    # 分割処理用のtemporary table 実行後drop

In [ ]:
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            smn_uid bigint
            ,advertising_id string
            ,drawbridge_consumer_id string
            ,user_agent_os_family string
            ,user_agent_device_family string
            ,user_agent_browser_family string
            ,user_agent_os_major string
            ,user_agent_os_minor string
            ,block int
            )
            
            partitioned by 
            (
            yymmdd string
            ,sample_size int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

In [ ]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert
for i in range(0,200):
    for d in range(1,2):
        with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                            request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
            query = (
                f"""
                insert overwrite table {uid_table}
                (
                smn_uid,
                advertising_id,
                drawbridge_consumer_id,
                user_agent_os_family,
                user_agent_device_family,
                user_agent_browser_family,
                user_agent_os_major,
                user_agent_os_minor,
                block
                )
                partition
                (
                yymmdd = to_date(days_add(now(),-{d}))
                ,sample_size = {i}
                )
                select
                distinct t1.smn_uid
                ,t1.advertising_id
                ,drawbridge_consumer_id
                ,user_agent_os_family
                ,user_agent_device_family
                ,user_agent_browser_family
                ,block
                from
                dsplog.bid_response t1
                inner join 
                segment.ya_samedevice_smn_uid_userlist_190723 t2
                on t1.smn_uid = t2.smn_uid
                where 
                concat_ws('-',year,month,day) between to_date(days_add(now(),-{d})) and to_date(days_add(now(),-{d})) 
                and user_agent_os_family in('Android','iOS')
                and t1.smn_uid is not null
                and t1.advertising_id is null
                and block = {i}
                """
            )
            ir.execute_query_no_ret(query) 

In [ ]:
query = (
    f"""
    select
    count(distinct smn_uid) as smn_uid_uu
    from 
    dm_tmp.ya_db_req_smnuidbase
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_tmp_uid = ir.sql_to_pandas(query)

In [ ]:
df_tmp_uid.head()

↓smn_uidが不明なidfaユーザーリストに対し、紐づけする

%%slack_notify @yusuke_akada
query = ( 
    f"""
    select
    count(distinct t1.advertising_id) as uu
    from 
    dm_tmp.ya_db_req_idfa t1
    inner join 
    dm_tmp.ya_db_req_smnuidbase t2
    on t1.drawbridge_consumer_id = t2.drawbridge_consumer_id
    and t1.user_agent_os_family = t2.user_agent_os_family
    and t1.user_agent_device_family = t2.user_agent_device_family
    and t1.yymmdd = t2.yymmdd
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_merge =  ir.sql_to_pandas(query)

In [ ]:
df_merge.head()

In [ ]:
df_tmp_adid.head()

In [ ]:
#1日分のマージだと46％（24342/52759）のみマッチング

In [ ]:
#1週間のマージだと54％（56056/102524）のみマッチング

セイムデバイス済みテーブルを作成する

In [ ]:
uid_table = "segment.samedevice_user_list_1906"   # idfaリストの最終的な対応table
temp_table = "dm_tmp.ya_samedevice"    # 分割処理用のtemporary table 実行後drop

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            advertising_id string
            ,smn_uid bigint
            ,drawbridge_consumer_id string
            )
            
            partitioned by 
            (
            block_no int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

In [ ]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert
for i in range(0,200):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            insert overwrite table {uid_table}
            (
            advertising_id,
            smn_uid,
            drawbridge_consumer_id
            )
            partition
            (
            block_no = {i}
            )
            select
            distinct t1.advertising_id
            ,t2.smn_uid
            ,t1.yymmdd
            from 
            dm_tmp.ya_db_req_idfa_190626 t1
            inner join 
            dm_tmp.ya_db_req_smnuid_190626 t2
            on t1.drawbridge_consumer_id = t2.drawbridge_consumer_id
            and t1.user_agent_os_family = t2.user_agent_os_family
            and t1.user_agent_device_family = t2.user_agent_device_family
            /*and t1.yymmdd = t2.yymmdd*/
            and t1.sample_size = {i}
            and t2.sample_size = {i}
            """
        )
        ir.execute_query_no_ret(query) 

In [ ]:
#2.上記1のユーザーをsmn_uidベースとanidベースでbid_responseを参照する。

idfa_idベースで

In [ ]:
uid_table = "dm_tmp.ya_db_req_idfa_190626"   # idfaリストの最終的な対応table
temp_table = "dm_tmp.ya_temp_db_sample"    # 分割処理用のtemporary table 実行後drop

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            advertising_id string
            ,smn_uid bigint
            ,drawbridge_consumer_id string
            ,user_agent_os_family string
            ,user_agent_device_family string
            ,user_agent_browser_family string
            ,block int
            )
            
            partitioned by 
            (
            yymmdd string
            ,sample_size int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

In [ ]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert
for i in range(0,200):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            insert overwrite table {uid_table}
            (
            advertising_id,
            smn_uid,
            drawbridge_consumer_id,
            user_agent_os_family,
            user_agent_device_family,
            user_agent_browser_family,
            block
            )
            partition
            (
            yymmdd = to_date(days_add(now(),-1))
            ,sample_size = {i}
            )
            select
            distinct t1.advertising_id
            ,t1.smn_uid
            ,drawbridge_consumer_id
            ,t1.user_agent_os_family
            ,t1.user_agent_device_family
            ,t1.user_agent_browser_family
            ,block
            from
            dsplog.bid_response t1
            inner join 
            dm_tmp.samedevice_adid_userlist_190621 t2
            on t1.advertising_id = t2.advertising_id
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-1)) and to_date(days_add(now(),-1)) 
            and user_agent_os_family in('Android','iOS')
            and is_app = True
            and t1.advertising_id is not null
            and t1.smn_uid is null
            and block = {i}
            """
        )
        ir.execute_query_no_ret(query) 

cookie_idベースで

In [ ]:
uid_table = "dm_tmp.ya_db_req_smnuid_190626"   # idfaリストの最終的な対応table
temp_table = "dm_tmp.ya_temp_db_sample"    # 分割処理用のtemporary table 実行後drop

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            smn_uid bigint
            ,advertising_id string
            ,drawbridge_consumer_id string
            ,user_agent_os_family string
            ,user_agent_device_family string
            ,user_agent_browser_family string
            ,block int
            )
            
            partitioned by 
            (
            yymmdd string
            ,sample_size int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

In [ ]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert
for i in range(0,200):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            insert overwrite table {uid_table}
            (
            smn_uid,
            advertising_id,
            drawbridge_consumer_id,
            user_agent_os_family,
            user_agent_device_family,
            user_agent_browser_family,
            block
            )
            partition
            (
            yymmdd = to_date(days_add(now(),-1))
            ,sample_size = {i}
            )
            select
            distinct t1.smn_uid
            ,t1.advertising_id
            ,drawbridge_consumer_id
            ,user_agent_os_family
            ,user_agent_device_family
            ,user_agent_browser_family
            ,block
            from
            dsplog.bid_response t1
            inner join 
            dm_tmp.samedevice_smn_uid_userlist_190621 t2
            on t1.smn_uid = t2.smn_uid
            where 
            concat_ws('-',year,month,day) between to_date(days_add(now(),-1)) and to_date(days_add(now(),-1)) 
            and user_agent_os_family in('Android','iOS')
            and t1.smn_uid is not null
            and t1.advertising_id is null
            and block = {i}
            """
        )
        ir.execute_query_no_ret(query) 

セイムデバイス済みテーブルを作成する

In [ ]:
uid_table = "segment.samedevice_user_list_190626"   # idfaリストの最終的な対応table
temp_table = "dm_tmp.ya_samedevice"    # 分割処理用のtemporary table 実行後drop

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            advertising_id string
            ,smn_uid bigint
            ,drawbridge_consumer_id string
            ,yymmdd string
            )
            
            partitioned by 
            (
            block_no int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

In [ ]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert
for i in range(0,200):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
            f"""
            insert overwrite table {uid_table}
            (
            advertising_id,
            smn_uid,
            drawbridge_consumer_id,
            yymmdd
            )
            partition
            (
            block_no = {i}
            )
            select
            distinct t1.advertising_id
            ,t2.smn_uid
            ,t1.drawbridge_consumer_id
            ,t1.yymmdd
            from 
            dm_tmp.ya_db_req_idfa_190626 t1
            inner join 
            dm_tmp.ya_db_req_smnuid_190626 t2
            on t1.drawbridge_consumer_id = t2.drawbridge_consumer_id
            and t1.user_agent_os_family = t2.user_agent_os_family
            and t1.user_agent_device_family = t2.user_agent_device_family
            and t1.yymmdd = t2.yymmdd
            and t1.sample_size = {i}
            and t2.sample_size = {i}
            """
        )
        ir.execute_query_no_ret(query) 